<a href="https://colab.research.google.com/github/dtarrio/captura-y-almacenamiento-de-la-informacion/blob/main/TP2DiegoTarrio2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Captura y Almacenamiento de la Información 2024
## Trabajo práctico 2 (Trabajando con información en JSon y MongoDB)

**Alumno:**

**Diego Fernando Tarrío**

*Licenciado en Informática U.N.L.P.*




---


### Información de los archivos json

**Ventas_ultima_campana.json:** Contiene array con 4 atributos:
*  id_vendedora: numeric
*  id_coordinadora: numeric
*  cantidad: numeric
*  monto: float

e.g. de elemento del array:

```
#  {
    "id_vendedora": 8173,
    "id_coordinadora": 138,
    "cantidad": 8,
    "monto": 299.843826657088
  }
```

---

**puntaje_acumulado.json:** Contiene array con 2 atributos:
*  id_vendedora: numeric
*  puntaje_acumulado: numeric

e.g. de elemento del array:

```
# {
    "id_vendedora": 18462,
    "puntaje_acumulado": 1051
  }
```






# Preparación ambiente

In [ ]:
# Instalación de MongoDB
!apt update
!apt install wget curl gnupg2 software-properties-common apt-transport-https ca-certificates lsb-release
!curl -fsSL https://www.mongodb.org/static/pgp/server-6.0.asc|sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/mongodb-6.gpg
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu $(lsb_release -cs)/mongodb-org/6.0 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!apt update
!apt install mongodb-org
!mkdir /data
!mkdir /data/db
!mongod --fork --logpath /var/log/mongodb/mongod.log

!pip install pymongo

In [ ]:
import json
import re
import pymongo
from pymongo import MongoClient

## Check DB server status
!mongod --version

print ("MongoDB is running. Version:", pymongo.version)

In [ ]:
# Creo una conexión con el servidor usando la url de conexión default.
url_server = 'localhost:27017'
client = MongoClient(url_server)

# Creamos la base de datos vendotutti
db = client.vendotutti

# Listo todas las bases de datos que existen en el servidor MongoDB (sólo para chequear)
print(client.list_database_names())

# Creación de la base de datos
## Lee ambos archivos json y los carga en respectivas colecciones dentro de la BD

In [8]:
# Leo el archivo json para puntaje_acumulado

db.puntaje_acumulado.delete_many({}) #limpio la collection puntaje_acumulado por posibles ejecuciones previas

json_filename = "puntaje_acumulado.json"

json_file = open(json_filename, "r")

puntaje_acumulado = json.load(json_file)

# Recorro json de puntaje_acumulado

for puntaje in puntaje_acumulado:
    punt = {
      "id_vendedora" : puntaje["id_vendedora"],
      "puntaje_acumulado" : puntaje["puntaje_acumulado"]
    }
    db.puntaje_acumulado.insert_one(punt)

# como alternativa al for previo podría utilizar la opción insert_many de pymongo:
# db.puntaje_acumulado.insert_many(puntaje_acumulado)
# igual lo dejo con el for por si fuese necesario ajustar algún atributo manualmente previo a su inserción en la colección,
# adicionalmente evito manejar lotes en el caso que el volumen exceda el límite de tamaño para insertar

In [9]:
# Leo el archivo json para ventas_ultima_campana

db.ventas_ultima_campana.delete_many({}) #limpio la collection ventas_ultima_campana por posibles ejecuciones previas

json_filename = "ventas_ultima_campana.json"

json_file = open(json_filename, "r")

ventas_ultima_campana = json.load(json_file)

# Recorro json de puntaje_acumulado

for venta in ventas_ultima_campana:
    vent = {
      "id_vendedora" : venta["id_vendedora"],
      "id_coordinadora" : venta["id_coordinadora"],
      "cantidad" : venta["cantidad"],
      "monto" : venta["monto"]
    }
    db.ventas_ultima_campana.insert_one(vent)

# como alternativa al for previo podría utilizar la opción insert_many de pymongo:
# db.ventas_ultima_campana.insert_many(ventas_ultima_campana)
# igual lo dejo con el for por si fuese necesario ajustar algún atributo manualmente previo a su inserción en la colección,
# adicionalmente evito manejar lotes en el caso que el volumen exceda el límite de tamaño para insertar

In [17]:
#chequeo básico de las colections en la db
print("Cantidad de documentos en puntaje_acumulado: ",db.puntaje_acumulado.count_documents({}))
print("Cantidad de documentos en ventas_ultima_campana: ",db.ventas_ultima_campana.count_documents({}))

Cantidad de documentos en puntaje_acumulado:  16853
Cantidad de documentos en ventas_ultima_campana:  119970


# Actualización de la base de datos con el cálculo del sueldo de cada vendedora

criterio:
sueldov = 1000 x puntos_acumuladosv x bonus_personalv x bonus_coordinadora



#OPCIÓN 1: Mayor trabajo en código python trayendo las colecciones a memoria y trabajando fuera de MongoDB (luego se implementa misma solución pero priorizando el trabajo directo en MongoDB mediante agregaciones en OPCIÓN 2):

In [ ]:
from collections import defaultdict

# Obtengo puntajes acumulados
puntajes_acumulados = db.puntaje_acumulado.find({}) #traigo toda la collection de puntaje_acumulado desde la bd

# Obtengo ventas de la última campaña
ventas_ultima_campana = db.ventas_ultima_campana.find({}) #traigo toda la collection de ventas_ultima_campana desde la bd

# Calculo el total de productos vendidos y el dinero recaudado por cada vendedora
ventas_por_vendedora = defaultdict(lambda: {'total_productos': 0, 'total_dinero': 0, 'id_coordinadora': None})

for venta in ventas_ultima_campana:
    id_vendedora = venta['id_vendedora']
    ventas_por_vendedora[id_vendedora]['total_productos'] += venta['cantidad']
    ventas_por_vendedora[id_vendedora]['total_dinero'] += venta['monto']
    ventas_por_vendedora[id_vendedora]['id_coordinadora'] = venta['id_coordinadora']

# Calculo bonus personal según la cantidad de productos vendidos
def calcular_bonus_personal(total_productos):
    if total_productos > 500:
        return 2
    elif total_productos > 250:
        return 1.6
    elif total_productos > 50:
        return 1.3
    elif total_productos > 10:
        return 1.1
    elif total_productos > 0:
        return 1.02
    else:
        return 1

# Calculo el total de dinero recaudado por cada coordinadora
dinero_por_coordinadora = defaultdict(float)
for id_vendedora, datos in ventas_por_vendedora.items():
    dinero_por_coordinadora[datos['id_coordinadora']] += datos['total_dinero'] #acumulo el total de la vendedora para la posición del diccionario que corresponde a la coordinadora asociada

# Ordeno coordinadoras por la cantidad de dinero recaudado
coordinadoras_ordenadas = sorted(dinero_por_coordinadora.items(), key=lambda x: x[1], reverse=True)

# Asigno bonus por coordinadora según la posición
bonus_coordinadora = {}
for idx, (id_coordinadora, _) in enumerate(coordinadoras_ordenadas):
    if idx == 0:
        bonus_coordinadora[id_coordinadora] = 1.35
    elif idx == 1:
        bonus_coordinadora[id_coordinadora] = 1.3
    elif idx == 2:
        bonus_coordinadora[id_coordinadora] = 1.25
    elif idx == 3:
        bonus_coordinadora[id_coordinadora] = 1.2
    elif idx == 4:
        bonus_coordinadora[id_coordinadora] = 1.15
    else:
        bonus_coordinadora[id_coordinadora] = 1.1

# Calculo el sueldo de cada vendedora
sueldos = {}
for puntaje in puntajes_acumulados:
    id_vendedora = puntaje['id_vendedora']
    puntaje_acumulado = puntaje['puntaje_acumulado']
    total_productos = ventas_por_vendedora[id_vendedora]['total_productos']
    id_coordinadora = ventas_por_vendedora[id_vendedora]['id_coordinadora']

    bonus_personal = calcular_bonus_personal(total_productos)
    bonus_coordinadora_vendedora = bonus_coordinadora.get(id_coordinadora, 1)

    sueldo = 1000 * puntaje_acumulado * bonus_personal * bonus_coordinadora_vendedora
    sueldos[id_vendedora] = sueldo

# Muestro los sueldos calculados
for id_vendedora, sueldo in sueldos.items():
    print(f"Vendedora {id_vendedora}: Sueldo = ${sueldo:.2f}")

# Guardo los sueldos de cada vendedora en otra colección en la BD:
db.sueldos_clientSide.delete_many({})
db.sueldos_clientSide.insert_many([{'id_vendedora': id_vendedora, 'sueldo': sueldo} for id_vendedora, sueldo in sueldos.items()])

# Opción 2: Llevando la mayor parte de la lógica a scripts sobre MongoDB (Este enfoque aprovecha las capacidades de agregación de la BD, reduciendo la carga en el lado del cliente):

In [ ]:

#Calculo el bonus personal y lo persisto en una colección en la db:

# Limpio la colection bonus_personal antes de realizar cualquier cálculo (para blanquear cualquier ejecución previa)
db.bonus_personal.delete_many({})

db.bonus_personal.insert_many(list(db.ventas_ultima_campana.aggregate([
    {
        "$group": {
            "_id": "$id_vendedora",
            "total_productos": {"$sum": "$cantidad"},
            "id_coordinadora": {"$first": "$id_coordinadora"}
        }
    },
    {
        "$addFields": {
            "bonus_personal": {
                "$switch": {
                    "branches": [
                        {"case": {"$gt": ["$total_productos", 500]}, "then": 2},
                        {"case": {"$gt": ["$total_productos", 250]}, "then": 1.6},
                        {"case": {"$gt": ["$total_productos", 50]}, "then": 1.3},
                        {"case": {"$gt": ["$total_productos", 10]}, "then": 1.1},
                        {"case": {"$gt": ["$total_productos", 0]}, "then": 1.02}
                    ],
                    "default": 1
                }
            }
        }
    }
])
))

#chequeo básico de la nueva collection bonus_personal en la db:
print("Cantidad de documentos en bonus_personal: ",db.bonus_personal.count_documents({}))

# Limpio la colection bonus_coordinadora antes de realizar cualquier cálculo (para blanquear cualquier ejecución previa)
db.bonus_coordinadora.delete_many({})

#Calculo el bonus de coordinadora y lo persisto en una colección en la bd:
db.bonus_coordinadora.insert_many(list( db.ventas_ultima_campana.aggregate([
    {
        "$group": {
            "_id": "$id_coordinadora",
            "total_dinero": {"$sum": "$monto"}
        }
    },
    {
        "$sort": {"total_dinero": -1}  # Ordenar de mayor a menor recaudación
    },
    {
        "$setWindowFields": {
            "sortBy": {"total_dinero": -1}, #Si bien ya se ordenó por mismo criterio en stage previo, la función nueva de MongoDB (setWindowFields)
                                            #requiere que se aplique el orden explícitamente (aunque ya estuviera aplicado en la colección)

            "output": {
                "ranking": {"$rank": {}}  # Asigno un ranking basado en la recaudación
            }
        }
    },
    {
        "$addFields": {
            "bonus_coordinadora": {
                "$switch": {
                    "branches": [
                        {"case": {"$eq": ["$ranking", 1]}, "then": 1.35},
                        {"case": {"$eq": ["$ranking", 2]}, "then": 1.3},
                        {"case": {"$eq": ["$ranking", 3]}, "then": 1.25},
                        {"case": {"$eq": ["$ranking", 4]}, "then": 1.2},
                        {"case": {"$eq": ["$ranking", 5]}, "then": 1.15}
                    ],
                    "default": 1.1
                }
            }
        }
    }
])
))

#chequeo básico de la nueva collection bonus_coordinadora
print("Cantidad de documentos en bonus_coordinadora: ",db.bonus_coordinadora.count_documents({}))

#Agregación en MongoDB para calcular el sueldo:
sueldos = db.puntaje_acumulado.aggregate([
    {
        "$lookup": {
            "from": "bonus_personal",
            "localField": "id_vendedora",
            "foreignField": "_id",
            "as": "bonus_personal_info"
        }
    },
    {
        "$unwind": "$bonus_personal_info" #descompongo el array en documentos individuales para facilitar acceso en siguientes stages
    },
    {
        "$lookup": {
            "from": "bonus_coordinadora",
            "localField": "bonus_personal_info.id_coordinadora",
            "foreignField": "_id",
            "as": "bonus_coordinadora_info"
        }
    },
    {
        "$unwind": "$bonus_coordinadora_info" #descompongo el array en documentos individuales para facilitar acceso en siguientes stages
    },
    {
        "$addFields": {
            "sueldo": {
                "$multiply": [
                    1000,
                    "$puntaje_acumulado",
                    "$bonus_personal_info.bonus_personal",
                    "$bonus_coordinadora_info.bonus_coordinadora"
                ]
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "id_vendedora": 1,
            "sueldo": 1
        }
    }
])



# Guardo sueldos en una colección de MongoDB
db.sueldos_via_db.delete_many({})
db.sueldos_via_db.insert_many(sueldos)

#chequeo básico de la nueva collection sueldos_via_db
print("Cantidad de documentos en sueldos_via_db: ",db.sueldos_via_db.count_documents({}))

#imprimo los resultados de la collection sueldos_via_db
for sueldo in db.sueldos_via_db.find():
    print(sueldo)

In [ ]:
# Realizo la agregación de los resultados de ambas opciones (sueldos_clientSide y sueldos_via_db) para unir ambas colecciones en función de id_vendedora
# de esta manera comparo resultados entre ambas soluciones:
resultados = db.sueldos_clientSide.aggregate([
    {
        "$lookup": {
            "from": "sueldos_via_db",
            "localField": "id_vendedora",
            "foreignField": "id_vendedora",
            "as": "resultados_comparados"  # El resultado de la unión se almacenará aquí
        }
    },
    {
        "$unwind": "$resultados_comparados"  # Desenrolla el array resultante de la unión
    },
    {
        "$limit": 20  # Limita el número de documentos a 20 ya que alcanza para validar que los cálculos se realizaron similares
    }
])

# Itero sobre los resultados para imprimir los datos comparados
for resultado in resultados:
    print(f"Vendedora ID: {resultado['id_vendedora']}")
    print(f"Sueldo calculado vía opción 1: {resultado}")
    print(f"Sueldo calculado vía opción 2: {resultado['resultados_comparados']}")
    print("-" * 50)
